# Latency and Tick

## Python Imports

In [1]:
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt

In [2]:
from scipy.optimize import minimize

In [3]:
from functools import partial

In [4]:
from numba import jit
from numba import njit
from numba import types
from numba.typed import Dict
# from numba.typed import List

## Tick Imports

In [5]:
# %matplotlib inline

In [5]:
from tick.hawkes import SimuHawkes, SimuHawkesMulti
from tick.base import TimeFunction
from tick.hawkes import HawkesKernelTimeFunc, HawkesKernelExp
# from tick.hawkes import HawkesKernel0
# from tick.hawkes import HawkesKernelSumExp
# from tick.hawkes import HawkesEM, HawkesSumExpKern
# from tick.hawkes import SimuPoissonProcess, SimuInhomogeneousPoisson
# from tick.hawkes import HawkesBasisKernels, HawkesSumGaussians
# from tick.plot import plot_timefunction
# from tick.plot import plot_point_process
# from tick.plot import plot_hawkes_kernels
# from tick.plot import plot_basis_kernels

In [ ]:
import hawklat as hl

## Definitions

In [7]:
n_paths = 100

In [8]:
end_time_1 = 100
end_time_2 = 1000
end_time_3 = 10000

## Simple example of exponential Kernel

In [7]:
α1 = 0.6
β1 = 0.8
λ01 = 1.2

In [8]:
def ek1(t):
    return α1 * np.exp(-β1 * t)

In [7]:
sup1 = 10

In [11]:
ekc1 = hl.time_funclat(ek1, sup1, 0)

In [12]:
ektf1 = HawkesKernelTimeFunc(ekc1)

In [13]:
ekprm1 = HawkesKernelExp(α1 / β1, β1)

In [14]:
# plot_timefunction(ekc1)

In [17]:
hawkestf1 = SimuHawkes(baseline=[λ01], kernels=[[ektf1]], end_time=end_time_1, verbose=False, seed=13)
hawkestf2 = SimuHawkes(baseline=[λ01], kernels=[[ektf1]], end_time=end_time_2, verbose=False, seed=13)
hawkestf3 = SimuHawkes(baseline=[λ01], kernels=[[ektf1]], end_time=end_time_3, verbose=False, seed=13)

In [18]:
multitf1 = SimuHawkesMulti(hawkestf1, n_simulations=n_paths)
multitf2 = SimuHawkesMulti(hawkestf2, n_simulations=n_paths)
multitf3 = SimuHawkesMulti(hawkestf3, n_simulations=n_paths)

In [19]:
multitf1.simulate()
multitf2.simulate()
multitf3.simulate()

In [20]:
hawkesprm1 = SimuHawkes(baseline=[λ01], kernels=[[ekprm1]], end_time=end_time_1, verbose=False, seed=13)
hawkesprm2 = SimuHawkes(baseline=[λ01], kernels=[[ekprm1]], end_time=end_time_2, verbose=False, seed=13)
hawkesprm3 = SimuHawkes(baseline=[λ01], kernels=[[ekprm1]], end_time=end_time_3, verbose=False, seed=13)

In [21]:
multiprm1 = SimuHawkesMulti(hawkesprm1, n_simulations=n_paths)
multiprm2 = SimuHawkesMulti(hawkesprm2, n_simulations=n_paths)
multiprm3 = SimuHawkesMulti(hawkesprm3, n_simulations=n_paths)

In [22]:
multiprm1.simulate()
multiprm2.simulate()
multiprm3.simulate()

In [9]:
def_bounds = [(0.01, 2.0), (0.01, 100.), (0.01, 200.)]

In [10]:
def_constraint = {'type': 'ineq', 'fun': lambda x:  x[2] - x[1]}

In [11]:
def_x0 = np.array([1., 0.3, 0.5])

In [32]:
%%time

np.mean(hl.findθ(multiprm1.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 520 ms, sys: 10.5 ms, total: 530 ms
Wall time: 535 ms


array([1.40146408, 0.59475211, 0.87366986])

In [33]:
%%time

np.mean(hl.findθlat(multiprm1.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 1.47 s, sys: 12.2 ms, total: 1.48 s
Wall time: 1.49 s


array([1.40146618, 0.59475306, 0.87367198])

In [34]:
%%time

np.mean(hl.findθ(multitf1.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 506 ms, sys: 7.1 ms, total: 513 ms
Wall time: 521 ms


array([1.40909889, 0.63330341, 0.87605308])

In [35]:
%%time

np.mean(hl.findθlat(multitf1.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 1.57 s, sys: 12 ms, total: 1.58 s
Wall time: 1.6 s


array([1.40909789, 0.63330295, 0.87605143])

In [36]:
%%time

np.mean(hl.findθ(multiprm2.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 959 ms, sys: 5.73 ms, total: 964 ms
Wall time: 992 ms


array([1.2216195 , 0.59841902, 0.80240309])

In [37]:
%%time

np.mean(hl.findθlat(multiprm2.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 11 s, sys: 25.6 ms, total: 11.1 s
Wall time: 11.3 s


array([1.22161187, 0.59841781, 0.80240005])

In [38]:
%%time

np.mean(hl.findθ(multitf2.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 1.1 s, sys: 6.57 ms, total: 1.1 s
Wall time: 1.13 s


array([1.22882478, 0.62591747, 0.80715263])

In [39]:
%%time

np.mean(hl.findθlat(multitf2.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 12.6 s, sys: 23.9 ms, total: 12.6 s
Wall time: 12.8 s


array([1.22883382, 0.62591863, 0.80715575])

In [40]:
%%time

np.mean(hl.findθ(multiprm3.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 5.91 s, sys: 19.7 ms, total: 5.93 s
Wall time: 6.02 s


array([1.20246849, 0.60137458, 0.80254908])

In [41]:
%%time

np.mean(hl.findθlat(multiprm3.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 2min 22s, sys: 409 ms, total: 2min 22s
Wall time: 2min 23s


array([1.20246828, 0.60137339, 0.80254723])

In [42]:
%%time

np.mean(hl.findθ(multitf3.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0), axis=0)

CPU times: user 7.27 s, sys: 19.4 ms, total: 7.29 s
Wall time: 7.29 s


array([1.2018588 , 0.62456376, 0.80126099])

In [43]:
%%time

np.mean(hl.findθlat(multitf3.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=0), axis=0)

CPU times: user 3min, sys: 416 ms, total: 3min
Wall time: 3min 1s


array([1.20185489, 0.62456684, 0.80126349])

## Simple example of exponential Kernel with latency

### Latency > 0

In [12]:
sup1 = 10
lat1 = 2

In [18]:
ekc3 = hl.time_funclat(ek1, sup1, lat1)

In [19]:
ektf3 = HawkesKernelTimeFunc(ekc3)

In [20]:
# plot_timefunction(ekc3)

In [21]:
hawkestf3_1 = SimuHawkes(baseline=[λ01], kernels=[[ektf3]], end_time=end_time_1, verbose=False, seed=13)
hawkestf3_2 = SimuHawkes(baseline=[λ01], kernels=[[ektf3]], end_time=end_time_2, verbose=False, seed=13)
hawkestf3_3 = SimuHawkes(baseline=[λ01], kernels=[[ektf3]], end_time=end_time_3, verbose=False, seed=13)

In [22]:
multitf3_1 = SimuHawkesMulti(hawkestf3_1, n_simulations=n_paths)
multitf3_2 = SimuHawkesMulti(hawkestf3_2, n_simulations=n_paths)
multitf3_3 = SimuHawkesMulti(hawkestf3_3, n_simulations=n_paths)

In [23]:
multitf3_1.simulate()
multitf3_2.simulate()
multitf3_3.simulate()

In [51]:
%%time

np.mean(hl.findθlat(multitf3_1.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=lat1), axis=0)

CPU times: user 1.75 s, sys: 20 ms, total: 1.77 s
Wall time: 1.76 s


array([1.41312185, 0.59432338, 0.81423172])

In [52]:
%%time

np.mean(hl.findθlat(multitf3_2.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=lat1), axis=0)

CPU times: user 15.5 s, sys: 70.9 ms, total: 15.6 s
Wall time: 15.6 s


array([1.26282245, 0.61530363, 0.80368218])

In [53]:
%%time

np.mean(hl.findθlat(multitf3_3.timestamps,
                 bounds=def_bounds, constraints=def_constraint,
                 x0=def_x0, τ=lat1), axis=0)

CPU times: user 3min 42s, sys: 2.04 s, total: 3min 44s
Wall time: 3min 43s


array([1.2093905 , 0.61256644, 0.78899373])

In [28]:
%%time

a = hl.τtsi(multitf3_1.timestamps[0][0], 2)

CPU times: user 10.5 ms, sys: 2.86 ms, total: 13.3 ms
Wall time: 10.8 ms


In [29]:
%%time

a = hl.τtsi(multitf3_2.timestamps[0][0], 2)

CPU times: user 101 ms, sys: 12.4 ms, total: 114 ms
Wall time: 103 ms


In [30]:
%%time

a = hl.τtsi(multitf3_3.timestamps[0][0], 2)

CPU times: user 1.28 s, sys: 92.8 ms, total: 1.38 s
Wall time: 1.31 s
